In [53]:
import numpy as np
import pandas as pd
import os
import io
import re
from unidecode import unidecode

path = r"C:/Users/Avisia/Documents/Football Transfer Tool/"
os.chdir(path)
import warnings
warnings.filterwarnings("ignore")

In [54]:
players = pd.read_csv("data/whoscored_data_scrapping_v2.csv", sep=';')

In [55]:
players.shape

(18684, 31)

In [56]:
players.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,Fouls,OffsidesWon,Clear,HasBeenDribbled,Blocks,KeyPasses,Dribbles,Fouled,Dispossessed,BadControl
0,Mohamed Salah,15/06/1992,England,2020/2021,Liverpool,EPL,5,450,6,0,...,0.4,0,0.2,0.2,0,3,2.6,0.4,2.2,3.8
1,Mohamed Salah,15/06/1992,England,2019/2020,Liverpool,EPL,33(1),2888,19,10,...,0.5,0,0.1,0.5,0,1.8,1.5,0.5,2,2.6
2,Mohamed Salah,15/06/1992,England,2019/2020,Liverpool,UCL,8,732,4,2,...,1,0,0,0.3,0,1.8,3,0.4,1.8,4.4
3,Mohamed Salah,15/06/1992,England,2019,Egypt,ACoN,4,360,2,0,...,1.5,0,0,0.5,0,1.3,1.5,1,2.5,3.3
4,Mohamed Salah,15/06/1992,England,2018/2019,Liverpool,EPL,37(1),3262,22,8,...,0.7,0,0.1,0.3,0,1.8,2.1,0.8,3.1,2.9


In [57]:
players[['Apps','As Sub', 'to delete']] = players['Apps'].str.split('\(|\)', expand=True)

In [58]:
players.drop(['to delete'], axis=1, inplace=True)

In [59]:
players.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,OffsidesWon,Clear,HasBeenDribbled,Blocks,KeyPasses,Dribbles,Fouled,Dispossessed,BadControl,As Sub
0,Mohamed Salah,15/06/1992,England,2020/2021,Liverpool,EPL,5,450,6,0,...,0,0.2,0.2,0,3,2.6,0.4,2.2,3.8,None
1,Mohamed Salah,15/06/1992,England,2019/2020,Liverpool,EPL,33,2888,19,10,...,0,0.1,0.5,0,1.8,1.5,0.5,2,2.6,1
2,Mohamed Salah,15/06/1992,England,2019/2020,Liverpool,UCL,8,732,4,2,...,0,0,0.3,0,1.8,3,0.4,1.8,4.4,None
3,Mohamed Salah,15/06/1992,England,2019,Egypt,ACoN,4,360,2,0,...,0,0,0.5,0,1.3,1.5,1,2.5,3.3,None
4,Mohamed Salah,15/06/1992,England,2018/2019,Liverpool,EPL,37,3262,22,8,...,0,0.1,0.3,0,1.8,2.1,0.8,3.1,2.9,1


In [60]:
players.fillna(0, inplace=True)

In [61]:
players.drop_duplicates(inplace=True)

In [62]:
players.shape

(16581, 32)

### Jointure des stats réelles avec la base de données des stats fifa

In [63]:
fifa = pd.read_csv('data/players.csv')

In [65]:
fifa.head()

,id,birth_date,sofifa_id,player_url,short_name,long_name,age,height_cm,weight_kg,nationality,...,poste_,poste,note_perf,nom,player_name,player_extended_name,date_of_birth,height,weight,perf
0,158023,1987-06-24,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,170,72,Argentina,...,RW,ATT,93.0,messi,messi,Lionel Messi,1987-06-24,170.0,72.0,5.000000
1,190460,1992-02-14,190460,https://sofifa.com/player/190460/christian-eri...,C. Eriksen,Christian Dannemann Eriksen,27,181,76,Denmark,...,RM,ATT,83.0,eriksen,eriksen,Christian Eriksen,1992-02-14,181.0,76.0,4.462366
2,195864,1993-03-15,195864,https://sofifa.com/player/195864/paul-pogba/20...,P. Pogba,Paul Pogba,26,191,84,France,...,LDM,MDC,78.0,pogba,pogba,Paul Pogba,1993-03-15,191.0,84.0,4.756098
3,198219,1991-06-04,198219,https://sofifa.com/player/198219/lorenzo-insig...,L. Insigne,Lorenzo Insigne,28,163,59,Italy,...,LS,ATT,87.0,insigne,insigne,Lorenzo Insigne,1991-06-04,163.0,59.0,4.677419
4,201024,1991-06-20,201024,https://sofifa.com/player/201024/kalidou-kouli...,K. Koulibaly,Kalidou Koulibaly,28,187,89,Senegal,...,LCB,DC,81.0,koulibaly,koulibaly,Kalidou Koulibaly,1991-06-20,187.0,89.0,4.764706


In [66]:
from datetime import date

def calculate_age(born):
    born = pd.to_datetime(born, format='%Y-%m-%d')
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [67]:
players['Player Name'].str.split(' ')[0][-1]

'Salah'

In [72]:
# séparer le nom du joueur de son prénom
players['nom'] = players.apply(lambda x : unidecode(x['Player Name'].split(' ')[-1].lower()), axis=1)
players['Player_Name'] = players['Player Name'].apply(lambda x: unidecode(x.lower()))
players['Team_'] = players['Team'].apply(lambda x: unidecode(x.lower()) if not x.lower()=='barcelona' else 'fc barcelona')
players['Team_'] = players['Team'].apply(lambda x: unidecode(x.lower()) if not x.lower()=='tottenham' else 'tottenham hotspur')
players['Tournament_'] = players['Tournament'].apply(lambda x : unidecode(x.lower()) if not pd.isna(x) else '')

fifa['age'] = fifa['date_of_birth'].apply(lambda x: calculate_age(x) if not pd.isna(x) else '')
fifa['long_name'] = fifa['long_name'].str.replace('-',' ')
fifa['long_name'] = fifa['long_name'].apply(lambda x : unidecode(x.lower()))
fifa['short_name'] = fifa['short_name'].apply(lambda x : unidecode(x.lower()))
fifa['player_extended_name'] = fifa['player_extended_name'].apply(lambda x : unidecode(x.lower()) if not pd.isna(x) else '')
fifa['player_extended_name'] = fifa['player_extended_name'].str.replace('-',' ')

In [73]:
players.drop(players[players['Player Name']=='Undefined'].index, inplace=True)

In [74]:
# conversion des dates de naissanc en format date identique 
players.birth = pd.to_datetime(players.birth, format = '%d/%m/%Y')
fifa.birth_date = pd.to_datetime(fifa.birth_date, format='%Y-%m-%d')

In [75]:
players_stats = pd.merge(players, fifa, 
                left_on=['Player_Name', 'birth'], 
                right_on=['short_name', 'birth_date'], how='inner')

In [76]:
players_stats2 = pd.merge(players, fifa, 
                left_on=['Player_Name', 'birth'], 
                right_on=['player_name', 'birth_date'], how='inner')

In [77]:
players_stats3 = pd.merge(players, fifa, 
                left_on=['Player_Name','birth'], 
                right_on=['player_extended_name','birth_date'], how='inner')

In [78]:
players_stats4 = pd.merge(players, fifa, 
                left_on=['Player_Name', 'birth'], 
                right_on=['long_name','birth_date'], how='inner')

In [79]:
players_stats5 = pd.merge(players, fifa, 
                left_on=['nom', 'birth'], 
                right_on=['short_name','birth_date'], how='inner')

In [80]:
players_stats6 = pd.merge(players, fifa, 
                left_on=['nom', 'birth'], 
                right_on=['player_name','birth_date'], how='inner')

In [81]:
players_stats7 = pd.merge(players, fifa, 
                left_on=['Team_', 'birth'], 
                right_on=['club','birth_date'], how='inner')

In [82]:
players_end = pd.concat([players_stats, players_stats2, players_stats3, players_stats4, 
                         players_stats5, players_stats6, players_stats7])

In [83]:
players_end.drop_duplicates(inplace=True)

In [84]:
players_end.shape

(15115, 151)

In [43]:
players_end[(players_end.duplicated(['Player Name','Team', 'Mins']))].head(1)

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,poste,note_perf,nom_y,player_name,player_extended_name,league,date_of_birth,height,weight,perf
23,Fabinho,1993-10-23,England,2015/2016,Monaco,UEL,5,450,0,0,...,MDC,79.0,fabinho,fabinho,fabio henrique tavares,Premier League,1993-10-23,188.0,78.0,4.817073


In [88]:
players_end.to_csv('data/joueurs_avec_stats.csv', index=False)

## A voir pour finaliser le dataset pour bien faire les jointures

In [85]:
a = set(players['Player Name'])

b=set(players_end['Player Name'])

In [86]:
print(len(a))
print(len(b))
print(len(a) - len(b))

2020
1614
406


In [87]:
len(a - b)

406

In [49]:
fifa[fifa['short_name']=='h. son']['club']

14236    tottenham hotspur
Name: club, dtype: object

In [52]:
players_end[players_end['Player Name']=='Alexis Sanchez'].head(2)

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,poste,note_perf,nom_y,player_name,player_extended_name,league,date_of_birth,height,weight,perf
2735,Alexis Sanchez,1988-12-19,Italy,2020/2021,Inter,ISA,1,120,0,1,...,ATT,81.0,sanchez,sanchez,alexis sanchez,Serie A TIM,1988-12-19,169.0,62.0,4.354839
2736,Alexis Sanchez,1988-12-19,Italy,2019/2020,Inter,ISA,10,918,4,8,...,ATT,81.0,sanchez,sanchez,alexis sanchez,Serie A TIM,1988-12-19,169.0,62.0,4.354839


In [51]:
set(a) - set(b)

{'Aaron Hickey',
 'Abdallahi Mahmoud',
 'Abdou Harroui',
 'Abdoul Kader Bamba',
 'Abdoulaye Sidibe',
 'Abdul Wahab Ibrahim',
 'Adam Idah',
 'Adel Taarabt',
 'Adil Aouchiche',
 'Adil Auassar',
 'Adrian Cubas',
 'Adrian Lopez',
 'Adrian Szoke',
 'Adrien Truffert',
 'Ahmad Toure Ngouyamsa Nounchil',
 'Aimen Moueffek',
 'Al Musrati',
 'Alex Soares',
 'Alexis Trouillet',
 'Amadou Ciss',
 'Amar Catic',
 'Ander Capa',
 'Anderson Carvalho',
 'Andraz Sporar',
 'Andrea Adorante',
 'Andrei Ratiu',
 'Andros Townsend',
 'Angel Rodriguez',
 'Anibal Chala',
 'Ansu Fati',
 'Ante Palaversa',
 'Anthony Briancon',
 'Anthony D Alberto',
 'Antonio Mirante',
 'Antonio Raillo',
 'Antony Evans',
 'Aridane Hernandez',
 'Arjen Robben',
 'Armando Broja',
 'Arnaud Kalimuendo Muinga',
 'Aurelien Scheidler',
 'Bandiougou Fadiga',
 'Bart Vriends',
 'Bassala Sambou',
 'Bastos',
 'Batista Mendy',
 'Ben Johnson',
 'Bilal Ould Chikh',
 'Bono',
 'Borja Sainz',
 'Bosko Sutalo',
 'Boubacar Hanne',
 'Boy Kemper',
 'Branco v

In [329]:
players[players.Player_Name=='habib diallo']

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,KeyPasses,Dribbles,Fouled,Dispossessed,BadControl,As Sub,Player_Name,Team_,Tournament_,nom
15387,Habib Diallo,1995-06-18,France,2020/2021,Strasbourg,FL1,0,53,1,0,...,1,2,0,0,2,1,habib diallo,strasbourg,fl1,diallo
15388,Habib Diallo,1995-06-18,France,2020/2021,Metz,FL1,1,141,0,0,...,0.3,0,0.5,1.5,1.8,3,habib diallo,metz,fl1,diallo
15389,Habib Diallo,1995-06-18,France,2019/2020,Metz,FL1,26,2199,12,3,...,0.9,0.8,0.8,1.3,3.3,0,habib diallo,metz,fl1,diallo
15390,Habib Diallo,1995-06-18,France,2017/2018,Metz,FL1,0,14,0,0,...,0,0,0,0,1,1,habib diallo,metz,fl1,diallo
15391,Habib Diallo,1995-06-18,France,2016/2017,Metz,FL1,7,824,1,1,...,0.4,0.2,0.5,1.2,1.5,12,habib diallo,metz,fl1,diallo


In [330]:
fifa[(fifa.short_name == 'h. diallo')]

,id,birth_date,sofifa_id,player_url,short_name,long_name,age,height_cm,weight_kg,nationality,...,player_name,player_extended_name,league,date_of_birth,height,weight,poste_,poste,note_perf,perf
692,225293,1995-06-15,225293,https://sofifa.com/player/225293/habib-diallo/...,h. diallo,habib diallo,,186,79,Senegal,...,NaN,,,NaN,NaN,NaN,ST,ATT,65.0,3.494624


# BROUILLON

In [114]:
players_end.filter(like=('style_')).columns.sort_values()

Index([], dtype='object')

In [ ]:
col = ['style_ancre']

In [517]:
mask = np.where(players_end[['style_artiste','style_maestro']]==True)

In [562]:
cols = ['Player Name','Club','Apps','Mins','Goals', 'Assists', 'Rating','perf','style']

In [567]:
a['Player Name']

3          David Silva
8             Angelino
12     Andreas Pereira
13           Juan Mata
15             Willian
            ...       
616       Yusuf Yazici
635      Wylan Cyprien
654      Memphis Depay
784     Ryad Boudebouz
786       Wahbi Khazri
Name: Player Name, Length: 111, dtype: object

In [115]:
a[a['Player Name'].isin(['Angelino'])]

NameError: name 'a' is not defined

In [619]:
df = pd.DataFrame(dict(
    r=[1, 5, 2, 2, 3],
    theta=['processing cost','mechanical properties','chemical stability',
           'thermal stability', 'device integration']))

In [622]:
df

,r,theta
0,1,processing cost
1,5,mechanical properties
2,2,chemical stability
3,2,thermal stability
4,3,device integration
